In [1]:
#coding:utf-8
"""
python 3
tensorflow 1.1
matplotlib 2.02
"""
import tensorflow as tf
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import PIL

learning_rate = 0.0001
batch_size=5

train_data = np.zeros((165, 64*64))
train_labels = np.zeros((165,1), dtype= np.uint8)

for people in range(1,16):
    for face in range(1,12):
        if(people >= 10):
            path = './Yale2/subject%d_%d.bmp' % (people, face)
        else:
            path = './Yale2/subject0%d_%d.bmp' % (people, face)
        oriImage = PIL.Image.open(path)
        oriImage = oriImage.resize((64,64))
        imageArray = np.array(oriImage)
        imageArray = imageArray.T
        imageVec = np.reshape(imageArray, imageArray.shape[0] * imageArray.shape[1])
        # print(imageArray.shape)
        # print(imageBigDataORL.shape)
#             print((people - 1) * 10 + face)
        train_data[(people - 1) * 11 + face - 1] = imageVec
        train_labels[(people - 1) * 11 + face - 1] = people
    

print(train_data.shape)
print(train_labels.shape)
plt.figure()
plt.imshow(train_data[0].reshape([64,64]))

train_labels = train_labels - 1

np.random.seed(100)
train_data = np.random.permutation(train_data[50:,:])
np.random.seed(100)
train_labels = np.random.permutation(train_labels[50:])
test_data = train_data[0:50,:]
test_labels = train_labels[0:50]
np.random.seed(200)
test_data = np.random.permutation(test_data)
np.random.seed(200)
test_labels = np.random.permutation(test_labels)

#将标签转为one_hot类型
def label_to_one_hot(labels_dense, num_classes=15):
    num_labels = labels_dense.shape[0]
    print(num_labels)
    index_offset = np.arange(num_labels) * num_classes
    print(index_offset)
    labels_one_hot = np.zeros((num_labels, num_classes))
    print(labels_dense.ravel())
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

#将图片转为灰度图
def to4d(img):
    return img.reshape(img.shape[0],64,64,1).astype(np.float32)/255

train_data = to4d(train_data)
train_labels = label_to_one_hot(train_labels,15)
test_data = to4d(test_data)
test_labels = label_to_one_hot(test_labels,15)
plt.figure()
plt.imshow(train_data[0,:,:,0])
plt.title(np.argmax(train_labels[0]))

xs = tf.placeholder(tf.float32,[None,64,64,1])
ys = tf.placeholder(tf.float32,[None,15])
keep_prob = tf.placeholder(tf.float32)


(165, 4096)
(165, 1)
115
[   0   15   30   45   60   75   90  105  120  135  150  165  180  195
  210  225  240  255  270  285  300  315  330  345  360  375  390  405
  420  435  450  465  480  495  510  525  540  555  570  585  600  615
  630  645  660  675  690  705  720  735  750  765  780  795  810  825
  840  855  870  885  900  915  930  945  960  975  990 1005 1020 1035
 1050 1065 1080 1095 1110 1125 1140 1155 1170 1185 1200 1215 1230 1245
 1260 1275 1290 1305 1320 1335 1350 1365 1380 1395 1410 1425 1440 1455
 1470 1485 1500 1515 1530 1545 1560 1575 1590 1605 1620 1635 1650 1665
 1680 1695 1710]
[12 11  9  7 13 11  6  7  7  6 11  9  8  7 13  6 14 11  8 13  7  9 11  6
 10 11  5 13  5  7 14 13  7  6 11 13  6 10  5  4  5 14 12 11 11  8 10  8
 12  9 12  8 12 14  6 14 10 13 10  4  6  8  8  8  8 14 14  7  9 12 14 10
  6 12 11  9 13 10 10  8  9  5 14  9 12 12  5  4  7  4  7  4 12  5  6  9
 10  5 14  7  5 14 10  9 13  9 13  5  8 11 12 13 10  6  5]
50
[  0  15  30  45  60  75  90 105 120

In [2]:
conv1 = tf.layers.conv2d(inputs=xs,filters=32,kernel_size=2,strides=1,padding='same',activation=tf.nn.relu)
pool1 = tf.layers.max_pooling2d(conv1,pool_size=2,strides=2)
conv2 = tf.layers.conv2d(pool1,filters=72,kernel_size=2,strides=1,padding='same',activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(conv2,pool_size=2,strides=2)
flat = tf.reshape(pool2,[-1,16*16*72])
dense = tf.layers.dense(flat,512,tf.nn.relu)
dropout = tf.nn.dropout(dense,keep_prob)
output = tf.layers.dense(dropout,15)

#计算loss
loss = tf.losses.softmax_cross_entropy(onehot_labels=ys,logits=output)
train = tf.train.AdamOptimizer(learning_rate).minimize(loss)
#返回两个参数一个train_opt,一个acc
accuracy = tf.metrics.accuracy(labels=tf.argmax(ys,axis=1),predictions=tf.argmax(output,axis=1))[1]

sess = tf.Session()
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
sess.run(init)
for step in range(122):
    i = 0
    while i < len(train_data):
        start = i
        end = i+batch_size
        batch_x = np.array(train_data[start:end])
        batch_y = np.array(train_labels[start:end])
        _,c = sess.run([train,loss],feed_dict={xs:batch_x,ys:batch_y,keep_prob:0.75})
        i += batch_size
    if step % 1 ==0:
        acc = sess.run(accuracy,feed_dict={xs:test_data,ys:test_labels,keep_prob:1})
        print('= = = = = = > > > > > > ','step:',step,'loss: %.4f'%c,'accuracy: %.2f' %acc)

= = = = = = > > > > > >  step: 0 loss: 2.4145 accuracy: 0.38
= = = = = = > > > > > >  step: 1 loss: 2.1502 accuracy: 0.52
= = = = = = > > > > > >  step: 2 loss: 1.5251 accuracy: 0.61
= = = = = = > > > > > >  step: 3 loss: 1.3538 accuracy: 0.66
= = = = = = > > > > > >  step: 4 loss: 1.0377 accuracy: 0.70
= = = = = = > > > > > >  step: 5 loss: 0.9475 accuracy: 0.74
= = = = = = > > > > > >  step: 6 loss: 0.4376 accuracy: 0.77
= = = = = = > > > > > >  step: 7 loss: 0.5246 accuracy: 0.80
= = = = = = > > > > > >  step: 8 loss: 0.4454 accuracy: 0.82
= = = = = = > > > > > >  step: 9 loss: 0.2954 accuracy: 0.84
= = = = = = > > > > > >  step: 10 loss: 0.1381 accuracy: 0.85
= = = = = = > > > > > >  step: 11 loss: 0.1117 accuracy: 0.87
= = = = = = > > > > > >  step: 12 loss: 0.1213 accuracy: 0.88
= = = = = = > > > > > >  step: 13 loss: 0.1826 accuracy: 0.89
= = = = = = > > > > > >  step: 14 loss: 0.0352 accuracy: 0.89
= = = = = = > > > > > >  step: 15 loss: 0.1439 accuracy: 0.90
= = = = = = > > > 